In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      /usr/local/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [4]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [5]:
# Resizing
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [6]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [7]:
l1 = nn.Conv2d(1, nh, 5) # 1 input/channel, 32 outputs, 5 x 5 kernel

In [8]:
x = x_valid[:100] # grabbing first 100 for testing

In [9]:
x.shape

torch.Size([100, 1, 28, 28])

In [10]:
def stats(x): return x.mean(),x.std()

In [11]:
l1.weight.shape # 32 output filters, 1 input filter/channel, 5 x 5 size of our tensor. (5x5 - see CNN eg from part-1. Go through Part-1 again)

torch.Size([32, 1, 5, 5])

In [12]:
stats(l1.weight),stats(l1.bias)

((tensor(-0.0063, grad_fn=<MeanBackward0>),
  tensor(0.1153, grad_fn=<StdBackward0>)),
 (tensor(-0.0496, grad_fn=<MeanBackward0>),
  tensor(0.1127, grad_fn=<StdBackward0>)))

In [13]:
t = l1(x) #O/P of first layer

In [14]:
stats(t)

(tensor(-0.0655, grad_fn=<MeanBackward0>),
 tensor(0.6656, grad_fn=<StdBackward0>))

In [15]:
init.kaiming_normal_(l1.weight, a=1.) # For ReLU, we have no Leak. What is Leak? See OneNote. So in order to adjust for a non-relu Kaiming Init behavior, we make the leak(a) as = 1.
stats(l1(x))
# See the the maen and std dev are way better than the above

(tensor(-0.0711, grad_fn=<MeanBackward0>),
 tensor(1.0093, grad_fn=<StdBackward0>))

In [17]:
import torch.nn.functional as F

In [18]:
def f1(x,a=0): return F.leaky_relu(l1(x),a)

In [19]:
# we then use the Kaiming Init for ReLU (a=0) to initialize our weights
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.4412, grad_fn=<MeanBackward0>),
 tensor(0.9136, grad_fn=<StdBackward0>))

<h3>We will write out own Kaming Init

In [20]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x)) #remember the defaul PyTorch lib doesn't handle the ReLU case explicitly

(tensor(0.2191, grad_fn=<MeanBackward0>),
 tensor(0.4092, grad_fn=<StdBackward0>))

In [21]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [25]:
l1.weight[0,0].shape

torch.Size([5, 5])

In [30]:
# receptive field size
rec_fs = l1.weight[0,0].numel() # just taking l1.weight[0.0] to get the 5 x 5 dim
rec_fs

25

In [26]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [28]:
nf,ni,*_ = l1.weight.shape # number_of_filters, number_of_inputs
nf,ni

(32, 1)

In [31]:
fan_in  = ni*rec_fs #number_of_inputs * receptive_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [32]:
def gain(a): return math.sqrt(2.0 / (1 + a**2)) # Kaiming init gain formula

In [34]:
# The below will explain the leaky gain phenomenon where the value of a ranges from 1 to sqrt(5) which is as in the PyTorch Docs (cell 3 from the top): init.kaiming_uniform_(self.weight, a=math.sqrt(5)). One thing to note is that this accounts for a uniform kaiming and not a normal kaiming but still doesn't explain the .57 value (not sure why)
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [35]:
torch.zeros(10000).uniform_(-1,1).std() #Jeremy just took a random sample of 10k random numbers to see what the STD looks like for a UNIFORM dist

tensor(0.5751)

In [36]:
1/math.sqrt(3.)

0.5773502691896258

In [37]:
# Jeremy's Kaiming
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [38]:
kaiming2(l1.weight, a=0)
stats(f1(x)) # these values are so much closer to normality compared to the PyTorch native src 

(tensor(0.4921, grad_fn=<MeanBackward0>),
 tensor(0.9033, grad_fn=<StdBackward0>))

In [44]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x)) # Using the sqrt(5) to check if we get same values as the PyTorch library and we do. We did this in cells above using this:
# l1 = nn.Conv2d(1, nh, 5)
# stats(f1(x))

(tensor(0.2215, grad_fn=<MeanBackward0>),
 tensor(0.3884, grad_fn=<StdBackward0>))

We'll create a sample 4 layer CNN using PyTorch defaults and run to see how the output stats look like

In [45]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [55]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [47]:
y = y_valid[:100].float()

In [48]:
t = m(x)
stats(t) #not close to NORMAL at all

(tensor(-0.0442, grad_fn=<MeanBackward0>),
 tensor(0.0085, grad_fn=<StdBackward0>))

In [49]:
l = mse(t,y)
l.backward()

In [50]:
stats(m[0].weight.grad) # checking the stats n the weight and if you look at the variance, it is very low 

(tensor(0.0236), tensor(0.0326))

In [51]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only 
        used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in

In [64]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [53]:
t = m(x)
stats(t)

(tensor(-0.4908, grad_fn=<MeanBackward0>),
 tensor(0.2116, grad_fn=<StdBackward0>))

In [54]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0452), tensor(0.4491))

## Export

In [ ]:
!./notebook2script.py 02a_why_sqrt5.ipynb